In [2]:
# import spacy
# from spacy.tokenizer import Tokenizer
import nltk.data
import nltk
from nltk import word_tokenize
import re
from nltk import FreqDist
import pandas as pd
import re
import csv
import pickle
import itertools
import glob
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen
import math
import random
import json
from util import tokenizeAn

### Carregando os dados

In [3]:
dataset_treinamento = pd.read_csv("dataset_treinamento.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim', 'Documento'], encoding='utf-8')
print("Tamanho do dataset de treinamento",len(dataset_treinamento))
dataset_treinamento =dataset_treinamento[0:8000]

Tamanho do dataset de treinamento 129380


In [4]:
#Esse arquivo é utilizado para extrairmos as setenças a que cada token rotulado no dataset pertence. É utilizado pelo
# pela célula de código a seguir
with open("dataset_para_rotulacao_manual_versao_2.txt", "r", encoding='utf-8') as file:
    textos = file.read().splitlines()
print("Quantida de de sentenças no dataSet utilizado na rotulação manual:", len(textos))
textos = textos [0:10000]

Quantida de de sentenças no dataSet utilizado na rotulação manual: 13628


### Conveter os dados no formato IOB para uma lista 

Conveter os dados no formato IOB para uma lista contendo as sentenças e uma lista contendo as tuplas com as posições da entidade e a categoria de entidade

In [5]:
def converteIOBtoSpacy(dataset_treinamento):
    sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
    s = textos[sMarker]
    dataSetEntradaTreinamento = []
    entities = []
    for index, row in dataset_treinamento.iterrows():
        if(row['Sentenca']-1 != sMarker): # Nova sentença
            if entities:
                dataSetEntradaTreinamento.append((s, [tuple(e) for e in entities])) # Salva sentença anterior
            entities = [] # esvazia entidades
            sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
            if (sMarker < len(textos)): # Limite de textos
                s = textos[sMarker]
            
        if(row['Rotulo'][0] == 'O'):
             entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
        if(row['Rotulo'][0] == 'B'):
             entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
        if(row['Rotulo'][0] == 'I'):
            if (entities): 
                entities[-1][1] = row['Fim']
            else:
                # print(index)
                entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
            
        if index == dataset_treinamento.index[-1]: # Ultimo elemento
            if entities:
                dataSetEntradaTreinamento.append((s, [tuple(e) for e in entities]))
            entities = []    
    return dataSetEntradaTreinamento

In [6]:
datasetTreinamento = converteIOBtoSpacy(dataset_treinamento)
## print(datasetTreinamento,'\n')

### Cria uma lista com as sentenças a serem rotuladas

In [7]:
lista_sentencas = []

for index, item in enumerate(datasetTreinamento):
    sentenca = item[0]
    lista_sentencas.append(sentenca)
len(lista_sentencas)

594

In [8]:
## Remove sentenças duplicadas
lista_sentencas = list(dict.fromkeys(lista_sentencas))

In [9]:
dict_entidades_categorias = dict({})
for index, item in enumerate(datasetTreinamento):
    for dado in item[1]:
        if dado[2] != '':
            entidade = item[0][dado[0]:dado[1]]
            if entidade not in dict_entidades_categorias.keys():
                dict_entidades_categorias[entidade] = dado[2]
            elif dado[2] != dict_entidades_categorias[entidade]:
                print("A palavra '" + str(entidade) + "' possui mais de um rotulacao, sao elas: \n1- " + str(dict_entidades_categorias[entidade]) + "\n2- " + str(dado[2]))

A palavra 'Taverna de Minas' possui mais de um rotulacao, sao elas: 
1- NOME_BEBIDA
2- NOME_ORGANIZACAO
A palavra 'Velho Ferreira' possui mais de um rotulacao, sao elas: 
1- NOME_BEBIDA
2- NOME_PESSOA
A palavra 'frutado' possui mais de um rotulacao, sao elas: 
1- CARACTERISTICA_SENSORIAL_AROMA
2- CARACTERISTICA_SENSORIAL_SABOR
A palavra 'Sacca' possui mais de um rotulacao, sao elas: 
1- NOME_BEBIDA
2- NOME_ORGANIZACAO
A palavra 'SACCA' possui mais de um rotulacao, sao elas: 
1- NOME_BEBIDA
2- NOME_ORGANIZACAO
A palavra 'baunilha' possui mais de um rotulacao, sao elas: 
1- CARACTERISTICA_SENSORIAL_AROMA
2- CARACTERISTICA_SENSORIAL_SABOR
A palavra 'Salinas' possui mais de um rotulacao, sao elas: 
1- NOME_LOCAL
2- NOME_BEBIDA
A palavra 'amadeirado' possui mais de um rotulacao, sao elas: 
1- CARACTERISTICA_SENSORIAL_SABOR
2- CARACTERISTICA_SENSORIAL_AROMA
A palavra 'amadeirado' possui mais de um rotulacao, sao elas: 
1- CARACTERISTICA_SENSORIAL_SABOR
2- CARACTERISTICA_SENSORIAL_AROMA
A pal

O comando abaixo permite editar o texto da categoria de entidade nomeada. É usado para alterarmos a categoria de entidade nomeada para entidades com mais de uma categoria, haja vista que o código da linha acima seleciona a primeira entidade com sua respectiva categoria automaticamente.

Transforma os itens do dicionario em tuplas com pares de Entidade e Categoria

In [10]:
lista_nova = list(dict_entidades_categorias.items())

In [11]:
listaCategoriasExistentes = ['CARACTERISTICA_SENSORIAL_AROMA','CARACTERISTICA_SENSORIAL_CONSISTÊNCIA','CARACTERISTICA_SENSORIAL_SABOR', 'CARACTERISTICA_SENSORIAL_COR','RECIPIENTE_ARMAZENAMENTO','EQUIPAMENTO_DESTILACAO','CLASSIFICACAO_BEBIDA','TEMPO_ARMAZENAMENTO','GRADUACAO_ALCOOLICA','TIPO_MADEIRA','NOME_BEBIDA','VOLUME','NOME_LOCAL','NOME_ORGANIZACAO','NOME_PESSOA','PRECO','TEMPO'] 
listaFiltraPorCategorias = []
for item in listaCategoriasExistentes:
    listaFiltraPorCategorias.append(list(filter(lambda x: x[1] == item, lista_nova)))
listaCategoriasExistentes

['CARACTERISTICA_SENSORIAL_AROMA',
 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA',
 'CARACTERISTICA_SENSORIAL_SABOR',
 'CARACTERISTICA_SENSORIAL_COR',
 'RECIPIENTE_ARMAZENAMENTO',
 'EQUIPAMENTO_DESTILACAO',
 'CLASSIFICACAO_BEBIDA',
 'TEMPO_ARMAZENAMENTO',
 'GRADUACAO_ALCOOLICA',
 'TIPO_MADEIRA',
 'NOME_BEBIDA',
 'VOLUME',
 'NOME_LOCAL',
 'NOME_ORGANIZACAO',
 'NOME_PESSOA',
 'PRECO',
 'TEMPO']

In [12]:
lista_final = [[] for i in range(10)]

for item in listaFiltraPorCategorias:
    porcentagem_10 = math.ceil(len(item)*0.1)
    random.shuffle(item)
    for index, y in enumerate(range(0, len(item), porcentagem_10)):
        porcentagem_10 = porcentagem_10 if porcentagem_10<len(item) else len(item)
        lista_final[index] += item[0:porcentagem_10]
        del item[0:porcentagem_10]

### Algoritmo que realiza a rotulação automatica

In [13]:
## Metodo para rotular as entidades
def rotulacao_entidades(lista_sentencas, pattern, dict_respostas): 
    t = [] ## lista que conterá as rotulações automaticas
    for sentenca in lista_sentencas:
        resultado = re.finditer(pattern, sentenca) 
        lista_aux = []
        for iterator in resultado:
            lista_aux.append((iterator.start(), iterator.end(), dict_respostas[iterator.group()]))
        t.append(dict({'data': sentenca, 'label': lista_aux}))
    return tokenizeAn(t)

In [ ]:
pattern = r""
for index, lista in enumerate(lista_final):
    pattern += "|".join(r"\b" + str(re.escape(x[0])) + r"\b" for x in lista)
    t = rotulacao_entidades(lista_sentencas, pattern, dict_entidades_categorias)
    with open("Resultado_com_"+str((index+1)*10)+"_porcento.bin", "wb",encoding='utf-8') as arquivo:
        for item in t:
            arquivo.write("%s\n" % str(item))
        print('Done')